In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
df = df.sample(20000)

In [4]:
df.head()

,review,sentiment
41621,Did I step in something or is that bad smell c...,negative
20384,Legendary director Sidney Lumet gives us one o...,positive
2050,Well - when the cameo appearance of Jason Mill...,negative
8091,"The first point that calls the attention in ""F...",negative
14947,Killer Flood: The Day the Damn Broke: 1/10: Fi...,negative


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 41621 to 33649
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     20000 non-null  object
 1   sentiment  20000 non-null  object
dtypes: object(2)
memory usage: 468.8+ KB


In [6]:
df.shape

(20000, 2)

In [7]:
import re
clean = re.compile('<.*?>')
re.sub(clean, '',df.iloc[2].review)

"Well - when the cameo appearance of Jason Miller (looking even more eroded than he did in Exorcist IV) is the high point of a picture, what've you got?It's a little bit country, a little bit rock n' roll: mix two drunks with money who drag their kid all over the place with a bog-dried mummy (have you figured that one out yet - DRIED in a bog?) in the basement, Christopher Walken with a bad dye job, and a little girl who might have been an interesting character if they'd developed her.I understand - sort of - that they're going back to visit her relatives. After that....Problem: There are several interesting flashbacks to what I must assume is her mother being killed in a car bombing (I think). This is never connected to anything. Problem: What do we need the grandmother for? Now, the grandmother could be interesting. She speaks Gaelic, or Celtic, or something. Maybe you can make something of her. The best they can do is that she 's got a tobacco habit. That's all.Problem: They cast a 

In [8]:
def cleanHtml(text):
    clear = re.compile('<.*?>')
    return re.sub(clean, '',text)

In [9]:
df['review'] = df['review'].apply(cleanHtml)

In [10]:
def lower_text(text):
    return text.lower()

In [11]:
df['review'] = df['review'].apply(lower_text)

In [12]:
def remove_special(text):
    x = ""
    for i in text:
        if i.isalnum():
            x = x+i
        else:
            x = x+' '
    return x   

In [13]:
df['review'] = df['review'].apply(remove_special)

In [14]:
import nltk

In [15]:
from nltk.corpus import stopwords

In [16]:
def remove_stopwords(text):
    x = []
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y = x[:]
    x.clear()
    return y

In [17]:
df['review'] = df['review'].apply(remove_stopwords)

In [18]:
df

,review,sentiment
41621,"[step, something, bad, smell, coming, daybreak...",negative
20384,"[legendary, director, sidney, lumet, gives, us...",positive
2050,"[well, cameo, appearance, jason, miller, looki...",negative
8091,"[first, point, calls, attention, ever, mozart,...",negative
14947,"[killer, flood, day, damn, broke, 1, 10, final...",negative
...,...,...
19643,"[meltdown, opens, scene, scientists, preparing...",negative
4732,"[wife, mental, health, therapist, watched, beg...",positive
49738,"[badly, acted, two, character, comedy, drama, ...",negative
6363,"[never, seen, movie, bad, even, entertaining, ...",negative


In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [20]:
def stem_words(text):
    y = []
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    return z

In [21]:
df['review'] = df['review'].apply(stem_words)

In [22]:
df

,review,sentiment
41621,"[step, someth, bad, smell, come, daybreak, 1, ...",negative
20384,"[legendari, director, sidney, lumet, give, us,...",positive
2050,"[well, cameo, appear, jason, miller, look, eve...",negative
8091,"[first, point, call, attent, ever, mozart, abs...",negative
14947,"[killer, flood, day, damn, broke, 1, 10, final...",negative
...,...,...
19643,"[meltdown, open, scene, scientist, prepar, con...",negative
4732,"[wife, mental, health, therapist, watch, begin...",positive
49738,"[badli, act, two, charact, comedi, drama, abru...",negative
6363,"[never, seen, movi, bad, even, entertain, enou...",negative


In [23]:
def join_back(list_in):
    return " ".join(list_in)

In [24]:
df['review'] = df['review'].apply(join_back)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [26]:
X = cv.fit_transform(df['review']).toarray()

In [27]:
X.shape

(20000, 48658)

In [28]:
y = df.iloc[:,-1]

In [29]:
y.shape

(20000,)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [31]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16000, 48658)
(4000, 48658)
(16000,)
(4000,)


In [32]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [33]:
cl1 = GaussianNB()
cl2 = MultinomialNB()
cl3 = BernoulliNB()

In [34]:
cl1.fit(X_train,y_train)
cl2.fit(X_train,y_train)
cl3.fit(X_train,y_train)

BernoulliNB()

In [35]:
y_pred1 = cl1.predict(X_test)
y_pred2 = cl2.predict(X_test)
y_pred3 = cl3.predict(X_test)

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
print(accuracy_score(y_test,y_pred1))
print(accuracy_score(y_test,y_pred2))
print(accuracy_score(y_test,y_pred3))

0.64275
0.8455
0.82775
